# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:

city_df = pd.read_csv('../WeatherPy/cities.csv')
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,74.19,83,100,12.50,PF,1650780222
1,1,Kropotkin,45.4375,40.5756,58.95,64,71,11.27,RU,1650780223
2,2,George Town,5.4112,100.3354,89.53,70,20,10.36,MY,1650780050
3,3,Sisimiut,66.9395,-53.6735,27.81,96,100,5.50,GL,1650780224
4,4,Teluknaga,-6.0989,106.6381,94.10,70,20,4.61,ID,1650780225


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Store latitude and longitude in locations
locations = city_df[['Lat', 'Lng']]

humidity = city_df['Humidity']

In [14]:
# Plot heatmap
city_map = gmaps.figure()


# Adding heatlayer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
city_map.add_layer(heat_layer)
city_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
filtered_citydf = city_df.loc[(city_df['Max Temp']>70) & (city_df['Max Temp'] < 80) & (city_df['Wind Speed'] < 10) & (city_df['Cloudiness'] < 1)]
filtered_citydf.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
99,99,Houma,29.5958,-90.7195,73.00,87,0,8.05,US,1650780305
126,126,Barra de Tecoanapa,16.5167,-98.7500,79.48,74,0,5.97,MX,1650780328
211,211,São Filipe,14.8961,-24.4956,73.36,74,0,5.82,CV,1650780399
219,219,Calhoun,42.2506,-85.0005,70.11,38,0,6.91,US,1650780405
254,254,Vila Velha,-20.3297,-40.2925,78.75,91,0,6.91,BR,1650780351


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df=filtered_citydf.copy()
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
99,99,Houma,29.5958,-90.7195,73.00,87,0,8.05,US,1650780305,
126,126,Barra de Tecoanapa,16.5167,-98.7500,79.48,74,0,5.97,MX,1650780328,
211,211,São Filipe,14.8961,-24.4956,73.36,74,0,5.82,CV,1650780399,
219,219,Calhoun,42.2506,-85.0005,70.11,38,0,6.91,US,1650780405,
254,254,Vila Velha,-20.3297,-40.2925,78.75,91,0,6.91,BR,1650780351,
334,334,Fort Myers Beach,26.4520,-81.9481,72.01,78,0,8.05,US,1650780490,
391,391,Ojinaga,29.5667,-104.4167,79.75,17,0,3.44,MX,1650780546,
395,395,Nioro,13.3500,-15.7500,79.07,30,0,4.18,GM,1650780550,
443,443,Gunjur,13.2019,-16.7339,71.67,60,0,4.61,GM,1650780595,
508,508,Murgab,37.4966,61.9714,76.84,38,0,4.41,TM,1650780650,


In [17]:
# params dictionary to update each iteration
params={
    'radius' : 5000,
    'types' : 'hotel',
    'keyword' : 'hotel',
    'key' : g_key,
}

for index, row in hotel_df.iterrows():
    city = row['City']
    
    lat = row['Lat']
    lng = row['Lng']

    params['location'] = f'{lat},{lng}'

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'


    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()

    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_name['results'][0]['name']
    except (KeyError, IndexError):
        print(f'Missing result for {city}. Skipping..') 
        hotel_df.loc[index, 'Hotel Name'] = 'NA' 

Missing result for Nioro. Skipping..
Missing result for Murgab. Skipping..


In [18]:
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
99,99,Houma,29.5958,-90.7195,73.00,87,0,8.05,US,1650780305,"Holiday Inn Houma, an IHG Hotel"
126,126,Barra de Tecoanapa,16.5167,-98.7500,79.48,74,0,5.97,MX,1650780328,Hotel Garapacho
211,211,São Filipe,14.8961,-24.4956,73.36,74,0,5.82,CV,1650780399,La Fora Ecolodge
219,219,Calhoun,42.2506,-85.0005,70.11,38,0,6.91,US,1650780405,"Holiday Inn Express Marshall, an IHG Hotel"
254,254,Vila Velha,-20.3297,-40.2925,78.75,91,0,6.91,BR,1650780351,Hotel Santorini


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:

marker= gmaps.marker_layer(locations, info_box_content=hotel_info)
city_map.add_layer(marker)

city_map

Figure(layout=FigureLayout(height='420px'))